<a href="https://colab.research.google.com/github/malloyca/CSC581B/blob/main/Final%20Project/small_convnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notes

- Complete trial with eight layer model
- Add more feedback to early stopping
  - accuracy and loss for that epoch
- Ideas for improving the performance

In [1]:
# imports
import torch
from torch import nn
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision import datasets
from torchvision.transforms import ToTensor, Compose, Normalize
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import copy
import numpy as np


In [2]:
# Load the training data (CIFAR10 to start)
training_data = datasets.CIFAR100(
    root = "data",
    train = True,
    download = True,
    transform = Compose([ToTensor(),
                        Normalize([0.5071, 0.4867, 0.4408],
                                  [0.2675, 0.2565, 0.2761])])
)

# Load the test data
test_data = datasets.CIFAR100(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor()
)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
training_targets = training_data.targets

In [4]:
train_split_index, valid_split_index = train_test_split(
    np.arange(len(training_targets)), test_size=0.2, stratify=training_targets
)

In [5]:
batch_size = 100

# Create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size,
                              sampler=SubsetRandomSampler(train_split_index))
valid_dataloader = DataLoader(training_data, batch_size=batch_size,
                              sampler=SubsetRandomSampler(valid_split_index))
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [6]:
# Check that it is splitting the data properly
train_length = 0
for _, y in train_dataloader:
  train_length += len(y)
print(f"Length of training split: {train_length}")

valid_length = 0
for _, y in valid_dataloader:
  valid_length += len(y)
print(f"Length of validation split: {valid_length}")

test_length = 0
for _, y in test_dataloader:
  test_length += len(y)
print(f"Length of test split: {test_length}")

Length of training split: 40000
Length of validation split: 10000
Length of test split: 10000


In [7]:
# Check that there are 100 instances of each class in the validation set
count = 0
for _, y in valid_dataloader:
  for target in y:
    if int(target.numpy()) == 0:
      count += 1

print(count)

100


# Building basic convolutional neural nets

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [29]:
# Define the model
class basicConvNet(nn.Module):
  def __init__(self, name):
    super(basicConvNet, self).__init__()
    self.name = name
    self.basic_conv_net = nn.Sequential(
        # 1st layer
        nn.Conv2d(3, 256, (3,3), padding='same'),
        nn.ReLU(),
        # 2nd layer
        nn.Conv2d(256, 128, (3,3), padding='same'),
        nn.ReLU(),
        nn.MaxPool2d(2),
        # 3rd layer - 16x16
        nn.Conv2d(128, 64, (3,3), padding='same'),
        nn.ReLU(),
        # 4th layer
        nn.Conv2d(64, 32, (3,3), padding='same'),
        nn.ReLU(),
        nn.MaxPool2d(2),
        # 5th layer - 8x8
        nn.Conv2d(32, 32, (3,3), padding='same'),
        nn.ReLU(),
        # 6th layer
        nn.Conv2d(32, 16, (3,3), padding='same'),
        nn.ReLU(),
        nn.MaxPool2d(2),
        # 7th layer - 4x4
        nn.Conv2d(16, 16, (3,3), padding='same'),
        nn.ReLU(),
        # 8th layer
        nn.Conv2d(16, 16, (3,3), padding='same'),
        nn.ReLU(),
    )
    self.flatten = nn.Flatten()
    self.output = nn.Linear(16*4*4, 100)

  # Forward propagation
  def forward(self, x):
    x = self.basic_conv_net(x)
    x = self.flatten(x)
    x = self.output(x)
    return x
      

#### Six layer model

        # 1st layer
        nn.Conv2d(3, 256, (3,3), padding='same'),
        nn.ReLU(),
        nn.MaxPool2d(2),
        # 2nd layer
        nn.Conv2d(256, 128, (3,3), padding='same'),
        nn.ReLU(),
        nn.MaxPool2d(2),
        # 3rd layer
        nn.Conv2d(128, 64, (3,3), padding='same'),
        nn.ReLU(),
        nn.MaxPool2d(2),
        # 4th layer
        nn.Conv2d(64, 32, (3,3), padding='same'),
        nn.ReLU(),
        #nn.MaxPool2d((2,2)),
        # 5th layer
        nn.Conv2d(32, 32, (3,3), padding='same'),
        nn.ReLU(),
        #nn.MaxPool2d((2,2)),
        # 6th layer
        nn.Conv2d(32, 16, (3,3), padding='same'),
        nn.ReLU(),

In [10]:
# Training loop
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # Compute prediction error
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backprop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [11]:
# Test function
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
  return test_loss

In [12]:
n_epochs = 100

In [13]:
def training_loop(n_epochs, model, train_data, test_data, loss_function,
                  optimizer, early_stopping=False, patience=10):
  current_epoch = 0
  best_epoch = 0
  best_loss = float('inf')
  patience_counter = 0

  for e in range(n_epochs):
    print(f"\nEpoch {e+1}\n----------------------------")
    train(train_data, model, loss_function, optimizer)
    test_loss = test(test_data, model, loss_function)

    # Iterate epoch counter
    current_epoch += 1

    # If early_stopping check test_loss
    if early_stopping:
      # case: test loss beats the current best loss
      if test_loss < best_loss:
        # store loss
        best_loss = test_loss

        # reset patience counter
        patience_counter = 0

        # store model and epoch number
        print("Storing new best model.")
        best_model_state_dict = copy.deepcopy(model.state_dict)
        best_epoch = current_epoch

      # Case: patience limit not yet reached => iterate patience counter
      elif patience_counter < patience - 1:
        patience_counter += 1
        print(f"Patience count: {patience_counter}")

      # Case: patience limit reached
      else:
        print("Finished due to early stopping.")
        print(f"Saving best model: {model.name}_epoch-{best_epoch:03d}")
        torch.save(best_model_state_dict, f'{model.name}_epoch-{best_epoch:03d}')
        break

  # If we get here, we did not stop early - save best model
  if early_stopping:
    print(f"Saving best model: {model.name}_epoch-{best_epoch:03d}")
    torch.save(best_model_state_dict, f'{model.name}_epoch-{best_epoch:03d}')

In [33]:
model = basicConvNet('eight-layer').to(device)

In [34]:
# Loss function
loss_fn = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)

In [35]:
training_loop(n_epochs, model, train_dataloader, valid_dataloader, loss_fn,
              optimizer, early_stopping=True)


Epoch 1
----------------------------
loss: 4.605928 [    0/50000]
loss: 4.608476 [10000/50000]
loss: 4.604364 [20000/50000]
loss: 4.603601 [30000/50000]
Test Error: 
 Accuracy: 0.2%, Avg loss: 4.605181 

Storing new best model.

Epoch 2
----------------------------
loss: 4.603875 [    0/50000]
loss: 4.604411 [10000/50000]
loss: 4.603657 [20000/50000]
loss: 4.596471 [30000/50000]
Test Error: 
 Accuracy: 0.5%, Avg loss: 4.527261 

Storing new best model.

Epoch 3
----------------------------
loss: 4.555882 [    0/50000]
loss: 4.468088 [10000/50000]
loss: 4.484347 [20000/50000]
loss: 4.400254 [30000/50000]
Test Error: 
 Accuracy: 1.2%, Avg loss: 4.222248 

Storing new best model.

Epoch 4
----------------------------
loss: 4.212809 [    0/50000]
loss: 4.226405 [10000/50000]
loss: 3.940852 [20000/50000]
loss: 3.950976 [30000/50000]
Test Error: 
 Accuracy: 1.9%, Avg loss: 3.939277 

Storing new best model.

Epoch 5
----------------------------
loss: 4.081413 [    0/50000]
loss: 4.139066 [1

KeyboardInterrupt: ignored

# Results on CIFAR100:

## Eight layer model

### Architecture
        # 1st layer
        nn.Conv2d(3, 256, (3,3), padding='same'),
        nn.ReLU(),
        # 2nd layer
        nn.Conv2d(256, 128, (3,3), padding='same'),
        nn.ReLU(),
        nn.MaxPool2d(2),
        # 3rd layer - 16x16
        nn.Conv2d(128, 64, (3,3), padding='same'),
        nn.ReLU(),
        # 4th layer
        nn.Conv2d(64, 32, (3,3), padding='same'),
        nn.ReLU(),
        nn.MaxPool2d(2),
        # 5th layer - 8x8
        nn.Conv2d(32, 32, (3,3), padding='same'),
        nn.ReLU(),
        # 6th layer
        nn.Conv2d(32, 16, (3,3), padding='same'),
        nn.ReLU(),
        nn.MaxPool2d(2),
        # 7th layer - 4x4
        nn.Conv2d(16, 16, (3,3), padding='same'),
        nn.ReLU(),
        # 8th layer
        nn.Conv2d(16, 16, (3,3), padding='same'),
        nn.ReLU(),

### Trials
- 

## Six layer model

### Architecture
- 32 x 32 x 256 conv, ReLU - (3,3), 256
- Maxpool (2,2)
- 16 x 16 x 128 conv, ReLU - (3,3), 128
- Maxpool (2,2)
- 8 x 8 x 64 conv, ReLU - (3,3), 64
- Maxpool (2,2)
- 4 x 4 x 32 conv, ReLU - (3,3), 32
- 4 x 4 x 16 conv, ReLU - (3,3), 16
- Flatten
- Fully connect layer (256, 100)

### Trials
- 7.3% / 2.499734
  - SGD
  - `lr = 5e-2`
  - Validation accuracy - 7.3% @ 20 epochs
  - Avg loss - 2.499734
- 7.4% / 2.490613
  - SGD
  - `lr = 1e-2`
  - Validation accuracy - 7.4% @ 65 epochs
  - Avg loss - 2.490613


# Results on CIFAR10:

## Six layer model (current best)
- Test accuracy: 72.7%
- Avg loss: 1.733851

Architecture:
- 32 x 32 x 256 conv, ReLU - (3,3), 256
- Maxpool (2,2)
- 16 x 16 x 128 conv, ReLU - (3,3), 128
- Maxpool (2,2)
- 8 x 8 x 64 conv, ReLU - (3,3), 64
- Maxpool (2,2)
- 4 x 4 x 32 conv, ReLU - (3,3), 32
- 4 x 4 x 16 conv, ReLU - (3,3), 16
- Flatten
- Fully connect layer (256, 10)